In [12]:
import streamlit as st
from encodec import EncodecModel
from encodec.utils import convert_audio
import torchaudio
import torch
from scipy.spatial.distance import cosine
import os
import pickle  # For storing pre-encoded songs

# Function to load and encode audio
def load_and_encode_audio(audio_file_path, model):
    wav, sr = torchaudio.load(audio_file_path)
    wav = convert_audio(wav, sr, model.sample_rate, model.channels)
    wav = wav.unsqueeze(0)
    with torch.no_grad():
        encoded_frames = model.encode(wav)
    codes = torch.cat([encoded[0] for encoded in encoded_frames], dim=-1)
    return codes

# Instantiate a pretrained EnCodec model
model = EncodecModel.encodec_model_24khz()
model.set_target_bandwidth(6.0)

def encode_file_batch():
    # Placeholder for storing encoded songs
    artist_encoded_songs = {}

    # Loop through the artist's songs
    for song_file in os.listdir("./audio_files/artist_main"):
        song_codes = load_and_encode_audio(f"./audio_files/artist_main/{song_file}", model)
        artist_encoded_songs[song_file]["codes"] = song_codes
        artist_encoded_songs[song_file]["artist"] = "Luke Combs"  # Change as needed
        # Isolate the song name
        song_name = song_file.split(".")[0]
        artist_encoded_songs[song_file]["song_name"] = song_name

    # Save the dictionary to a pickle file
    with open("encoded_artist_songs.pkl", "wb") as f:
        pickle.dump(artist_encoded_songs, f)

# Encode the artist's songs
encode_file_batch()


KeyError: 'lc_aint_far_from_it.mp3'

In [11]:
codes.shape

torch.Size([1, 8, 16165])